In [41]:
import pandas as pd
import json

In [3]:
tmap = pd.read_csv('../data/Tmap_2005.csv')

In [4]:
c1 = tmap['지역(시도)'] == '서울'
select_list = ['공공기관', '의료시설', '연구기관', '업종별기업', '행정기관', '종교', '가정의례']
# c2 = tmap['지역(시군구)'] == '강남구'
# c2 = tmap['일자']==20200510
seoul = tmap.loc[c1,:]
seoul.shape

(23250, 8)

In [5]:
l = []
for i in select_list:
    select_condition2 = tmap['검색지유형2']==i
    seoul_index = seoul.loc[select_condition2,:].index
    # l.append(seoul_index)
    l.extend(seoul_index)
new_seoul = seoul.drop(l)
new_seoul['검색지유형3'].unique()
# seoul['검색지유형2'].unique()
# seoul['검색지유형3'].unique()

array(['아파트브랜드', '지하철역', '아파트단지', '백화점', '아파트건설사', '공원', '할인점', '주상복합',
       '농수축산물시장', '대학교', '쇼핑센터', '테마파크', '주요빌딩', '일반상가', '산업시설기타',
       '고속도로 IC', '오피스텔', '스케이트장', '기차역', '전망대', '주차장', '박물관/기념관', '호텔',
       '전자제품양판점', '도서관', '극장', '커피전문점', '고등학교', '종합시장', '서적브랜드', '전문상가',
       '한식', '미술관', '드라이브코스', '문화유적지', '고궁', '버스터미널', '초등학교', '전시장',
       '수족관', '부페', '공항', '농수축산물점', '교차로', '학교내시설물', '유원지', '재래시장', '방송사',
       '운동장/체육관', '슈퍼마켓', '공연장', '섬', '분식', '기념비', '중학교', '아파트형공장',
       '종합스포츠센타', '골프장', '대한상공회의소', '은행', '주요건물기타', '주유소', '자동차영업소',
       '중고차매매', '대교/철교', '사회복지시설', '화랑/전시관', '삼성', '센터', '능묘', '회관',
       '도시고속화도로IC', '축제', '운전면허시험장', 'LG유플러스', '호수', '한샘가구', '관광안내소',
       '장난감', '동식물원', '패밀리레스토랑', '놀이시설'], dtype=object)

In [6]:
# 서울시 구
gu_list = new_seoul['지역(시군구)'].unique()
# 점수주기
new_seoul['point'] = 31-new_seoul['검색지랭킹(시군구내)']
# 지역별 점수 합산
result = pd.DataFrame()
for i in gu_list:
    con_gu = new_seoul['지역(시군구)'] == i
    # print(i)
    gu_seoul = new_seoul.loc[con_gu, :]
    s = gu_seoul.groupby(by=['검색지명']).sum()
    s = s.loc[s['point']>20,:]
    ss = s.sort_values(by='point', ascending=False).reset_index()
    ss['gu'] = i
    result=result.append(ss[['검색지명', 'point', 'gu']])
result


,검색지명,point,gu
0,고덕래미안힐스테이트아파트,930,강동구
1,천호역,904,강동구
2,고덕그라시움아파트,868,강동구
3,강동롯데캐슬퍼스트아파트,795,강동구
4,현대백화점천호점,770,강동구
...,...,...,...
26,오일뱅크가재울뉴타운셀프,55,서대문구
27,연세동문회관,49,서대문구
28,연희대우아파트,43,서대문구
29,돈의문센트레빌아파트,41,서대문구


## 카카오 api 연습
- 카카오 api 키워드로 장소검색을 사용하면 각각의 네비게이션 목적지들의 좌표값이 나온다.
- 지하철 역일경우에는 인구 분포가 안나오는데 그외에는 나온다.

In [7]:
name = result.iloc[0,0]

In [8]:
# cnt_df.plot(kind='bar')
import requests
from decouple import config
ka_key = config('KAKAO_RESTAPI')
kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='+name
h = {
    'Authorization': 'KakaoAK '+ka_key
}
res = requests.get(kakao_url, headers=h)
location_data = res.json()['documents']
# 구, 동 따로 구별해서 저장해주기
address = location_data[0]['address_name']
place_url = location_data[0]['place_url']
place_x = location_data[0]['x']
place_y = location_data[0]['y']
# print(place_x, place_y, address)
# location_data

In [27]:
kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CT1&x={place_x}&y={place_y}&radius=300&page=&sort=distance'
cate_res = requests.get(kakao_category_url, headers=h)
cate_res.json()

{'documents': [],
 'meta': {'is_end': True,
  'pageable_count': 0,
  'same_name': None,
  'total_count': 0}}

# 만들기

In [38]:
# 서울에 있는 내비게이션 목적지
result
gu_list = result['gu'].unique()
gu_list

array(['강동구', '성동구', '송파구', '양천구', '영등포구', '용산구', '종로구', '중랑구', '강남구',
       '강서구', '관악구', '광진구', '동작구', '마포구', '서초구', '은평구', '강북구', '구로구',
       '노원구', '도봉구', '동대문구', '성북구', '중구', '금천구', '서대문구'], dtype=object)

# 내비 목적지로 list만드는 부분

In [48]:
navi_list = []
for idx, gu_name in enumerate(gu_list):
    # if idx: break  ## test용으로
    con = result['gu'] == gu_name
    location = result.loc[con,:]
    for i in location['검색지명']:
        kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + i
        res = requests.get(kakao_url, headers=h)
        ### 내비게이션 목적지에서 많이 나오는 애들 항목임.
        location_data = res.json()['documents']
        try:

            place_x = location_data[0]['x']
            place_y = location_data[0]['y']
            navi_list.append(
                {
                    'name':i,
                    'x':place_x,
                    'y':place_y,
                }
            )
        except:
            print(res.json(),i)            

{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'same_name': {'keyword': '래미안옥수리버젠1단지아파트', 'region': [], 'selected_region': ''}, 'total_count': 0}} 래미안옥수리버젠1단지아파트
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'same_name': {'keyword': 'e편한세상옥수파크힐스1단지아파트', 'region': [], 'selected_region': ''}, 'total_count': 0}} E편한세상옥수파크힐스1단지아파트
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'sanme_name': None, 'total_count': 0}} 용산파크타워1단지아파트
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'same_name': {'keyword': '상봉종합버스터미널', 'region': [], 'selected_region': ''}, 'total_count': 0}} 상봉종합버스터미널
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'same_name': {'keyword': '동성12차아파트', 'region': [], 'selected_region': ''}, 'total_count': 0}} 동성12차아파트
{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'same_name': {'keyword': 'sk주유소행운셀프', 'region': [], 'selected_region': ''}, 'total_count': 0}} SK주유소행운셀프
{'documents': [

# 함수

In [39]:
def getinfo(l):
    re = []
    for i in l:
        a = i['address_name'].split()
        g, d = a[1], a[2]
        re.append(
            {
                'PlaceName':i['place_name'],
                'Gu':g,
                'Dong':d,
                'PlaceCategory':i['category_group_code'],
                'PlaceId': int(i['id']),
                'PlaceX':i['x'],
                'PlaceY':i['y'],
            }
        )
    return re

# 함수를 통해 내비목적지 반경 구하기
- 주변 300m
- 최대 45개 
- 겹치는거 제외하는 거는 구현못함 (메모리 할당 때문에)

In [49]:
category = ['FD6','CE7','CT1','AT4','MT1'] # 음식점, 카페, 문화시설, 관광명소, 대형마트
place_dict = {'Place':[]}
for place in navi_list:
    for cate in category:
        kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code={cate}&x={place["x"]}&y={place["y"]}&radius=300&page=1&sort=distance'
        cate_res = requests.get(kakao_category_url, headers=h).json()
        place_dict['Place'].extend(getinfo(cate_res['documents']))
        if 30<cate_res['meta']['pageable_count']:
            for p in range(2, 4):
                kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code={cate}&x={place["x"]}&y={place["y"]}&radius=300&page={p}&sort=distance'
                cate_res = requests.get(kakao_category_url, headers=h).json()
                place_dict['Place'].extend(getinfo(cate_res['documents']))
        elif 15<cate_res['meta']['pageable_count']:
            kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code={cate}&x={place["x"]}&y={place["y"]}&radius=300&page=2&sort=distance'
            cate_res = requests.get(kakao_category_url, headers=h).json()
            place_dict['Place'].extend(getinfo(cate_res['documents']))



In [50]:
len(place_dict['Place'])

46107

In [51]:
# json.dumps(place_dict, ensure_ascii=False, indent='\t')
with open('SeoulPlaceData.json','w',encoding='utf-8') as f:
    json.dump(place_dict, f,ensure_ascii=False, indent='\t')